# Random Events and how they affect the world.

At eny point `events.event_results(world)` will cause a random event to happen that will change the world in some way. __nothing__ is also a possible outcome. `world.culture.chaos` will determine the likelyhood of an event happening when this method is called. What you see here is what happens at the beginning of the game AND every so often when the character moves from place to place. 

In [1]:
import pandas as pd
import numpy as np
import os

import os
import sys
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()

#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import sys,pickle
sys.path.append('../')
from lib.builders import people,towns,nations, events,culture as c


with (open('world.p', 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

Random events happen to nations, although thier effects can change people and places. 

In [2]:
example_nation = world.nations[1]
example_nation

Nation of Helllight

In [3]:
key = world.get_filtered_chord(n=example_nation.name,
                   r='key',
                   t=['desert', 'forest', 'mountain', 'plain'])

world.df_features.loc[key]

y                       20
rainfall                11
x                       25
key                  25:20
elevation               26
terrain           mountain
feature                NaN
nation number            1
nation           Helllight
z                Helllight
danger               0.734
Name: 25:20, dtype: object

## An events table sorts the events and has all of the information needed to affect the world.
* A = The subject, or the nation that did the thing
* o = The object, or the nation that got the thing done to
* t = the terrain, place where the event affect is placed
* e = the eon (turn or year) that the event happened

In [4]:
events.get_possible_events()

['bandits', 'marrage', 'raiders', 'temple']

In [5]:
event = events.get_event(events.get_possible_events()[0])
event

<events: events object (bandits)>

The method `.event_results()` automatically chooses a random event and activates it. Any effect of the event should be envoked at this step. 

In [6]:
events.event_results(world)

['great_hall']


'{e}: a great temple was founded in the city of Kingsmoral in [Nation of Artspring]'

In [7]:
events.event_results(world)

['great_hall']


'{e}: a great temple was founded in the city of Stoneplace in [Nation of Earthridge]'

# Resolving the events


## Type one: Feature
A feature is placed in a part of the terrain.

Currently events don't happen at sea. This may change at some point. With panas as my core world database, this is easy as `df.loc[key,'feature'] = feature`. Rendering the feature is taken care of when the user travels to that place. 

In [8]:
def place_feature(world,event):
    for n in range(event.n_objects):
        n = np.random.choice(world.nations)
        key = world.get_filtered_chord(n=n.name,
                                       r='key',
                                      t=['desert', 'forest', 'mountain', 'plain'])
        print(key,":",event)
        world.df_features.loc[key,'feature'] = event.key
        #TODO do something with e (effect amount)
        return world.df_features.loc[key,'terrain']

place_feature(world,event)

29:4 : events object (bandits)


'mountain'

In [9]:
nation_with_bandits = world.df_features.loc[world.df_features['feature']=='bandits','nation'][0]
nation_with_bandits

'Waterblade'

In [10]:
world.df_features[(world.df_features['feature'].isnull()==False) &
                 (world.df_features['nation']==nation_with_bandits)]

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,z,danger
key,,,,,,,,,,,
31:1,1,2.0,31,31:1,23.0,town,Servantswish,0.0,Waterblade,City,1.029
32:1,1,0.0,32,32:1,28.0,town,Stonebrook,0.0,Waterblade,City,0.422
34:2,2,0.0,34,34:2,32.0,town,Hellkeep,0.0,Waterblade,City,0.691
29:4,4,11.0,29,29:4,29.0,mountain,bandits,0.0,Waterblade,Waterblade,0.657
33:4,4,0.0,33,33:4,36.0,town,Demonsblade,0.0,Waterblade,City,0.513


You can see that this feature has been added to this area.  

## Type Two: Favor of one nation is altered
Requires a __subject__ and an __object__. 

In [11]:
event = events.get_event('marrage')
event

<events: events object (marrage)>

In [12]:
event.effect_var, event.effect

('favor', Decimal('0.30'))

In [13]:
[(n.diplomacy) for n in world.nations][1]

{'Waterblade': {'favor': 0.4, 'stance': 'tense'},
 'Gleamridge': {'favor': 0.8, 'stance': 'peace'},
 'Demonslight': {'favor': 0.8, 'stance': 'peace'},
 'Earthridge': {'favor': 0.8, 'stance': 'peace'},
 'Landsspring': {'favor': 0.8, 'stance': 'peace'},
 'Artspring': {'favor': 0.8, 'stance': 'peace'},
 'Camtear': {'favor': 0.8, 'stance': 'peace'}}

In [14]:
def get_deplomacy_df(nation):
    d = pd.DataFrame(nation.diplomacy).T.reset_index(drop=False)
    d['nation'] = nation.name
    d.columns = ['neighbor','favor','stance','nation']
    return d[['nation','neighbor','favor','stance']]
    

def get_world_diplomacy(nations):
    diplomacy = pd.concat([get_deplomacy_df(n) for n in world.nations],sort=False).reset_index(drop=True)
    return diplomacy

get_world_diplomacy(world.nations)

# get_deplomacy_df(world.nations[0])

,nation,neighbor,favor,stance
0,Waterblade,Helllight,0.65,peace
1,Waterblade,Gleamridge,0.8,peace
2,Waterblade,Demonslight,0.8,peace
3,Waterblade,Earthridge,0.8,peace
4,Waterblade,Landsspring,0.8,peace
5,Waterblade,Artspring,0.8,peace
6,Waterblade,Camtear,0.8,peace
7,Helllight,Waterblade,0.4,tense
8,Helllight,Gleamridge,0.8,peace
9,Helllight,Demonslight,0.8,peace


# Cycling through many events is just a loop

In [15]:
for i in range(10):
    x = events.event_results(world)
    print(x.replace("{e}",str(i)))

['great_hall']
0: a great temple was founded in the city of Hellkeep in [Nation of Waterblade]
1: bandits have set up camp in the None of [Nation of Gleamridge]
2: raiders from [Nation of Landsspring], plundered [Nation of Earthridge, Nation of Demonslight, Nation of Helllight]
3: a noble from [Nation of Landsspring] was married to a noble in [Nation of Landsspring]
4: a noble from [Nation of Gleamridge] was married to a noble in [Nation of Artspring]
5: raiders from [Nation of Helllight], plundered [Nation of Artspring, Nation of Helllight, Nation of Earthridge]
['great_hall']
6: a great temple was founded in the city of Tearhail in [Nation of Demonslight]
['great_hall', 'temple']
7: a great temple was founded in the city of Buckwish in [Nation of Landsspring]
['great_hall', 'temple']
8: a great temple was founded in the city of Stoneplace in [Nation of Earthridge]
9: raiders from [Nation of Artspring], plundered [Nation of Camtear, Nation of Waterblade, Nation of Helllight]


Now I'll look into those events to see that they did get placed. 

In [16]:
np.unique(world.df_features['nation'].dropna())

array(['Artspring', 'Camtear', 'Demonslight', 'Earthridge', 'Gleamridge',
       'Helllight', 'Landsspring', 'Waterblade'], dtype=object)